# Detecting Money Laundering Patterns Across Global Financial Transactions- ETL process

## Objectives

This project will explore a large dataset of 10,000 records, containing information on international  financial transactions. The dataset includes categorical and numerical information as it relates to each financial transaction including country of origin, the legality of the transaction, transaction amount, industry, number of shell companies involved in a transaction, tax haven of country and risk rating. 

The objective is to determine what variable or group of variables within the dataset will determine if the transaction is illegal. To do this, I will:

- Carry out a complete ETL (Extract, Transform, Load) process to clean and prepare the data.

- Run statistical tests to evaluate the four different hypothesis.

- Use these insights to suggest money laundering risk reduction strategies.



## Ethics and Data Privacy

All personal and identifiable information has been thoroughly removed from the dataset to ensure the privacy and confidentiality of individuals and institutions involved. Specifically:
1. **Personal Identifiers:** Names, contact details, account numbers, and any direct personal identifiers have been fully anonymised. For example, individual names have been replaced with generic labels such as Person 123.
Institutional Data: Identifiable information relating to financial institutions, such as bank names or branch identifiers, has also been anonymised, using placeholders like Bank 4833.

2. **Indirect Identifiers:** Care has been taken to avoid inclusion of any combinations of data that could potentially lead to re-identification of individuals or organisations, in accordance with established data protection and ethical research guidelines.

3. **Data Handling:** The data has been stored and handled securely throughout the project, and access has been restricted to authorised personnel only.
These steps have been implemented to ensure compliance with ethical research standards and data protection regulations, including GDPR (where applicable), and to respect the privacy of all parties involved.

---

1. **Import Required Libraries**

I will begin by importing the necessary Python libraries for data handling and exploration.

In [53]:
import pandas as pd
import numpy as np
import os

# Step 0. Move up one directory level to where the dataset file is located, enabling pandas to find and load the CSV successfully
print(os.getcwd())

/Users/nataliewaugh/Documents/DataCode/Detecting_Money_Laundering_Patterns-/jupyter_notebooks


2. **Load the Dataset**

I will load the CSV file named money_laundering_dataset.csv from the local directory.

In [54]:
#Step 1. Load the dataset
data = pd.read_csv('/Users/nataliewaugh/Documents/DataCode/Detecting_money_laundering_patterns-/data/money_ laundering_dataset.csv')

#Step 2. show the first few rows of the dataset
data.head()

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country
0,TX0000000001,Brazil,3.267530e+06,Offshore Transfer,2013-01-01 00:00:00,Person_1101,Construction,USA,True,Illegal,6,1,Bank_40,Singapore
1,TX0000000002,China,4.965767e+06,Stocks Transfer,2013-01-01 01:00:00,Person_7484,Luxury Goods,South Africa,False,Illegal,9,0,Bank_461,Bahamas
2,TX0000000003,UK,9.416750e+04,Stocks Transfer,2013-01-01 02:00:00,Person_3655,Construction,Switzerland,True,Illegal,1,3,Bank_387,Switzerland
3,TX0000000004,UAE,3.864201e+05,Cash Withdrawal,2013-01-01 03:00:00,Person_3226,Oil & Gas,Russia,False,Illegal,7,2,Bank_353,Panama
4,TX0000000005,South Africa,6.433784e+05,Cryptocurrency,2013-01-01 04:00:00,Person_7975,Real Estate,USA,True,Illegal,1,9,Bank_57,Luxembourg


In [55]:
# Step 3. Display summary of dataset structure, including column names, non-null counts, and data types.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Transaction ID               10000 non-null  object 
 1   Country                      10000 non-null  object 
 2   Amount (USD)                 10000 non-null  float64
 3   Transaction Type             10000 non-null  object 
 4   Date of Transaction          10000 non-null  object 
 5   Person Involved              10000 non-null  object 
 6   Industry                     10000 non-null  object 
 7   Destination Country          10000 non-null  object 
 8   Reported by Authority        10000 non-null  bool   
 9   Source of Money              10000 non-null  object 
 10  Money Laundering Risk Score  10000 non-null  int64  
 11  Shell Companies Involved     10000 non-null  int64  
 12  Financial Institution        10000 non-null  object 
 13  Tax Haven Country

3. **Clean the Dataset**

I will check for irregularities across the dataset which may hinder further analysis i.e spelling mistakes and duplication. Moneytary values will be brought down to 0 decimal places, due to the large sums of money involved. 

In [56]:
# Step 4. Drop columns that are not needed for analysis, specifically the 'Transaction ID' column.
data.drop(columns=['Transaction ID'], inplace=True)

In [57]:
# Step 5. Check for duplicate values in 'Person Involved' column
print(data['Person Involved'].duplicated().sum())

3680


In [ ]:
# Step 5.1 Update name of country column to 'Origin Country'
data.rename(columns={'Country': 'Origin Country'}, inplace=True)
print(data.head())

  Origin Country  Amount (USD)   Transaction Type  Date of Transaction  \
0         Brazil  3.267530e+06  Offshore Transfer  2013-01-01 00:00:00   
1          China  4.965767e+06    Stocks Transfer  2013-01-01 01:00:00   
2             UK  9.416750e+04    Stocks Transfer  2013-01-01 02:00:00   
3            UAE  3.864201e+05    Cash Withdrawal  2013-01-01 03:00:00   
4   South Africa  6.433784e+05     Cryptocurrency  2013-01-01 04:00:00   

  Person Involved      Industry Destination Country  Reported by Authority  \
0     Person_1101  Construction                 USA                   True   
1     Person_7484  Luxury Goods        South Africa                  False   
2     Person_3655  Construction         Switzerland                   True   
3     Person_3226     Oil & Gas              Russia                  False   
4     Person_7975   Real Estate                 USA                   True   

  Source of Money  Money Laundering Risk Score  Shell Companies Involved  \
0         

In [65]:
print(data['Person Involved'].unique())

['Person_1101' 'Person_7484' 'Person_3655' ... 'Person_6348' 'Person_4171'
 'Person_3267']


Initially, I considered removing the `'Person Involved'` column because it contains coded identifiers. However, after discovering that there are 3,680 duplicate entries, it’s clear that this column provides valuable information for analysis. These repeated entries can help identify individuals involved in multiple transactions, which is important for detecting suspicious patterns or behaviors.

In [66]:
# Step 6. Check for duplicate values in 'Financial Institution' column
print(data['Financial Institution'].duplicated().sum())
data['Financial Institution'].value_counts()

9501


Financial Institution
Bank_81     36
Bank_260    36
Bank_100    35
Bank_120    34
Bank_438    33
            ..
Bank_199    11
Bank_169    11
Bank_269    11
Bank_249     9
Bank_133     9
Name: count, Length: 499, dtype: int64

The exploration above shows that there is repetition in the `'Financial Institution`' involved, with some banks appearing significantly more often than others. Therefore, I will retain this column to further analyse whether there is any correlation between the financial institution and the legality of the transactions.

In [67]:
#Step 7. Check for irregularies in 'Industry' column
print(data['Industry'].unique())

['Construction' 'Luxury Goods' 'Oil & Gas' 'Real Estate' 'Arms Trade'
 'Casinos' 'Finance']


`'Industry`' column contains seven industries and no irregularities in the data. 

In [68]:
#Step 8. Check for irregularities in remaining columns 
print(data['Reported by Authority'].unique())
print(data['Source of Money'].unique())
print(data['Destination Country'].unique())
print(data['Tax Haven Country'].unique())
 

[ True False]
['Illegal' 'Legal']
['USA' 'South Africa' 'Switzerland' 'Russia' 'Brazil' 'UK' 'India' 'China'
 'Singapore' 'UAE']
['Singapore' 'Bahamas' 'Switzerland' 'Panama' 'Luxembourg'
 'Cayman Islands']


The remaining columns in the dataset do not show any irregularities in their entries. There are a total of 10 countries where the source of money originates and six tax haven countries. Overall, the data quality in these columns appears to be good.

In [69]:
# Step 9. Change the monetary values to 0 decimal places
data['Amount (USD)'] = data['Amount (USD)'].round(0)
print(data['Amount (USD)'].head())
  

0    3267530.0
1    4965767.0
2      94168.0
3     386420.0
4     643378.0
Name: Amount (USD), dtype: float64


4. **Time Shift data**

The `Date of Transaction` in the dataset range from 2013 onwards. For the purpose of this analysis, the dates have been shifted forward by 10 years to align with the current timeline (2023 and beyond). This adjustment helps in contextualizing the data to present-day conditions without altering the relative timing of transactions, allowing for more relevant insights while maintaining data integrity.

In [70]:
# Step 10. Check date column is in the correct datetime format to allow for analysis 
data['Date of Transaction'] = pd.to_datetime(data['Date of Transaction'])

# Step 11.Shift the dates forward by 10 years
data['Date of Transaction'] = data['Date of Transaction'] + pd.DateOffset(years=10)
print(data['Date of Transaction'].head())

0   2033-01-01 00:00:00
1   2033-01-01 01:00:00
2   2033-01-01 02:00:00
3   2033-01-01 03:00:00
4   2033-01-01 04:00:00
Name: Date of Transaction, dtype: datetime64[ns]


Upon reviewing the dataset, I found that only three out of ten columns contain numerical values, while the remaining columns are categorical. A high proportion of categorical features can make it more challenging to visualize relationships and detect meaningful patterns. To address this, I will encode selected categorical columns as numerical or binary features, making the dataset more amenable to analysis and suitable for a broader range of visualizations and statistical techniques.

In [72]:
# Step 12. Check for unique values in categorical columns
print(data['Origin Country'].unique())
print(data['Destination Country'].unique())

['Brazil' 'China' 'UK' 'UAE' 'South Africa' 'Russia' 'Switzerland' 'India'
 'USA' 'Singapore']
['USA' 'South Africa' 'Switzerland' 'Russia' 'Brazil' 'UK' 'India' 'China'
 'Singapore' 'UAE']


## Feature Engineering 

There are 10 unique values in both the Country and Destination Country columns. There is overlap between the two columns, meaning some transactions may remain within the same country — whether legal or illegal. To enable clearer analysis in Tableau, especially for identifying cross-border risks and country-type behavior, I will create three engineered categorical columns:

1. **Transaction Type: Domestic vs Cross-Border**
Purpose: Quickly identifies if a transaction stays within the same country or goes across borders.
Method: Compare Country with Destination Country.


In [ ]:
# Step 13. Create new categorical column to easily identify whether the transaction that took place was domestic or cross-border.
data['Domestic or Cross-Border'] = np.where(data['Origin Country'] == data['Destination Country'],  
'Domestic', 'Cross-Border')
print(data['Domestic or Cross-Border'].value_counts())


Domestic or Cross-Border
Cross-Border    9013
Domestic         987
Name: count, dtype: int64


2. **Origin Country Category (Income & Regulation Bins)**
Based on UN classifications, World Bank income levels, and FATF regulatory status, countries can be grouped into five discrete bins:

| Origin Country Category          | Countries            |
|----------------------------------|----------------------|
| High Income / Regulated          | USA, UK, Switzerland |
| High Income / Tax Haven          | Singapore, UAE       |
| Upper-Middle / Regulated         | South Africa         |
| Upper-Middle / Partial Regulated | China, Brazil        |
| Lower-Middle / Partial Regulated | India                |
| Upper-Middle / High Risk         | Russia               |


In [81]:
#Step 13. Create Origin Country Category (Income & Regulation Bins)
# start by creating a dictionary to map countries to their income and regulation categories
country_category_map = {
    'USA': 'High Income / Regulated',
    'UK': 'High Income / Regulated',
    'Switzerland': 'High Income / Regulated',
    'Singapore': 'High Income / Tax Haven',
    'UAE': 'High Income / Tax Haven',
    'South Africa': 'Upper-Middle / Regulated',
    'China': 'Upper-Middle / Partial Regulated',
    'Brazil': 'Upper-Middle / Partial Regulated',
    'India': 'Lower-Middle / Partial Regulated',
    'Russia': 'Upper-Middle / High Risk',
}
# Step 14. Define the new column using apply + lambda
data['Origin Country Category'] = data['Origin Country'].apply(
    lambda x: country_category_map[x] if x in country_category_map else 'Unknown'
)

print(data['Origin Country Category'].value_counts())

Origin Country Category
High Income / Regulated             2972
Upper-Middle / Partial Regulated    2044
High Income / Tax Haven             1971
Upper-Middle / Regulated            1026
Upper-Middle / High Risk             997
Lower-Middle / Partial Regulated     990
Name: count, dtype: int64


---

save new clean dataset to my local drive. 

In [19]:
data.to_csv('/Users/nataliewaugh/Documents/DataCode/Detecting_Money_Laundering_Patterns-/data/Cleaned_Money_Laundering_Dataset.csv', index=False)